# 不均衡ラベル誘導についての調査

## (1) 概要

test_benefitone_conversation_mlp.pyにおいて、以下の通り、中途半端にfeatureが抽出された場合、不均衡ラベル[4678]に誘導されてしまう現象が見られました。

```
MacBookPro-makmorit-jp:learning makmorit$ nosetests -s learning/tests/engine/test_benefitone_conversation_mlp.py
:
======================================================================
FAIL: 抽出featureがある場合
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/makmorit/GitHub/donusagi-bot/learning/learning/tests/engine/test_benefitone_conversation_mlp.py", line 49, in test_dislike_carrot
    ok_(result.answer_id == Reply.CLASSIFY_FAILED_ANSWER_ID or result.probability < self.threshold)
AssertionError: None
:
learning.log: DEBUG: Reply#perform text_array.separated_sentences: ['ニンジン 嫌い 出す ない']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#perform features:   (0, 51)	0.707106781187
  (0, 281)	0.707106781187
learning.log: DEBUG: question: ニンジンが嫌いなので出さないでください
learning.log: DEBUG: question_feature_count: 2
learning.log: DEBUG: predicted results (order by probability desc)
learning.log: DEBUG: {'probability': 0.95676519513680003, 'answer_id': 4678.0}
learning.log: DEBUG: {'probability': 0.020753506184116418, 'answer_id': 0.0}
:
FAILED (failures=1)
```

こちらの原因等について詳細調査しましたところ、抽出された「ない」「出す」というfeatureから、それなりに高い重み付けで不均衡ラベル「4678」へ誘導されたことが確認されております。


- ない --(重み=0.783201)--> Unit:16 --(重み=0.421332)--> label:4678


- 出す --(重み=1.144951)--> Unit:83 --(重み=0.474521)--> label:4678

## (2) 抽出されたfeatureを確認

抽出されたfeatureと、そのTF-IDF値は以下の通りでした。

- (51, 'ない', 0.70710678118654746)


- (281, '出す', 0.70710678118654746)

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
'''
    学習ずみモデルを取得
    bot_id = 995
'''
from learning.core.persistance import Persistance

bot_id = 995
estimator = Persistance.load_model(bot_id)
vectorizer = Persistance.load_vectorizer(bot_id)

In [3]:
from learning.core.training_set.text_array import TextArray

questions = ['ニンジンが嫌いなので出さないでください']
text_array = TextArray(questions, vectorizer=vectorizer)
text_array.separated_sentences

2017/04/20 PM 04:39:10 TextArray#__init__ start


['ニンジン 嫌い 出す ない']

In [4]:
features = text_array.to_vec()
features

2017/04/20 PM 04:39:10 TextArray#to_vec start
2017/04/20 PM 04:39:10 TextArray#to_vec end


<1x644 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [5]:
def get_item_from_vocabulary(vocabulary, index):
    '''
        vocabulary から
        指定インデックスの単語を参照
    '''
    for k, v in vocabulary.items():
        if v == index:
            return k

    return None

def get_word_and_tfidfval_of_features(features, vocabulary):
    '''
        抽出featureのindexに対応する単語を、
        TF-IDF値と合わせて戻す
    '''
    features_list = []
    
    _, indices = features.nonzero()
    for index in indices:
        word = get_item_from_vocabulary(vocabulary, index)
        tfidfval = features[0, index]
        features_list.append((index, word, tfidfval))
        
    return features_list

In [6]:
get_word_and_tfidfval_of_features(features, vectorizer.vocabulary_)

[(51, 'ない', 0.70710678118654746), (281, '出す', 0.70710678118654746)]

## (3) MLPモデルの内容を確認

'ない' '出す' というfeatureから不均衡ラベル[4678]へ誘導する <a href="06-simplified.ipynb"><b>ユニット</b></a> を持っているかどうかを確認します。

生成されたMLPモデルの重み（影響変数＋バイアス）を確認し、上記テスト結果に至った裏付けを取ります。

In [7]:
HIDDEN_LAYER = 0
OUTPUT_LAYER = 1
threshold = 0.75 # 便宜的に決めています

index_dasu = 281 # (281, '出す')
index_nai  = 51  # (51, 'ない')

### (3-1) 中間ユニットの内容を確認

'ない' '出す' というfeatureから、どの中間ユニットへ誘導される可能性があるかを確認します。

In [8]:
import numpy as np
s = np.size(estimator.coefs_[HIDDEN_LAYER], axis=0) # <--- featureの数になります
u = np.size(estimator.coefs_[HIDDEN_LAYER], axis=1) # <--- ユニットの数になります
print ('Count: feature=%d, unit of hidden layer=%d' % (s, u))

Count: feature=644, unit of hidden layer=100


In [9]:
'''
     ユニットごとのバイアスを取得
'''
intercepts_unit = estimator.intercepts_[HIDDEN_LAYER]
np.size(intercepts_unit)

100

In [10]:
'''
     feature「ない」に対する、ユニットごとの重みを取得
'''
coefs_nai = estimator.coefs_[HIDDEN_LAYER][index_dasu]
omomi_nai = coefs_nai + intercepts_unit
indices_nai = np.where(omomi_nai > threshold) # 大きいものだけ抽出

In [11]:
'''
     feature「出す」に対する、ユニットごとの重みを取得
'''
coefs_dasu = estimator.coefs_[HIDDEN_LAYER][index_nai] 
omomi_dasu = coefs_dasu + intercepts_unit
indices_dasu = np.where(omomi_dasu > threshold) # 大きいものだけ抽出

In [12]:
'''
    feature「ない」から、どの中間ユニットへ誘導されやすかったか？
'''
list_nai = indices_nai[0]
for i in list_nai:
    print('index=%d, omomi=%0.6f' % (i, omomi_nai[i]))

index=16, omomi=0.783201
index=36, omomi=0.839417
index=62, omomi=0.785681
index=63, omomi=0.868256
index=97, omomi=0.751776


In [13]:
'''
    feature「出す」から、どの中間ユニットへ誘導されやすかったか？
'''
list_dasu = indices_dasu[0]
for i in list_dasu:
    print('index=%d, omomi=%0.6f' % (i, omomi_dasu[i]))

index=5, omomi=1.044044
index=6, omomi=0.797699
index=9, omomi=0.907592
index=15, omomi=1.074607
index=32, omomi=0.885518
index=50, omomi=0.769279
index=83, omomi=1.144951
index=85, omomi=0.962114
index=88, omomi=1.133707
index=95, omomi=0.807744
index=99, omomi=0.972738


### (3-2) 出力ユニットの内容を確認

どの中間ユニットから、不均衡ラベルへ誘導されやすいのかを確認します。

In [14]:
import numpy as np
s = np.size(estimator.coefs_[OUTPUT_LAYER], axis=0) # <--- 中間ユニットの数になります
u = np.size(estimator.coefs_[OUTPUT_LAYER], axis=1) # <--- 出力ユニット（＝ラベル）の数になります
print ('Count: unit of hidden layer=%d, label=%d' % (s, u))

Count: unit of hidden layer=100, label=85


In [15]:
'''
    出力レイヤーにおける
    不均衡ラベル「4678」のインデックスを取得
'''
index_imbalanced_label = np.where(estimator.classes_ == 4678)[0][0]
index_imbalanced_label

2

In [16]:
'''
     ラベルのバイアスを取得
'''
intercepts_imbalanced_label = estimator.intercepts_[OUTPUT_LAYER][index_imbalanced_label]
intercepts_imbalanced_label

0.137822537067083

In [17]:
'''
     不均衡ラベル「4678」に対する、ユニットごとの重みを取得
'''
coefs_imbalanced_label = estimator.coefs_[OUTPUT_LAYER].T[index_imbalanced_label]
omomi_imbalanced_label = coefs_imbalanced_label + intercepts_imbalanced_label
indices_imbalanced_label = np.where(omomi_imbalanced_label > 0.4) # 大きいものだけ抽出

不均衡ラベルへ誘導されやすい中間ユニットが数多く存在しており、かつ、重みが突出した中間ユニットがありません。

これは不均衡ラベルのサンプル数が多いため（＝多数のfeatureに紐づいているため）と考えております。

In [18]:
'''
    どの中間ユニットから、不均衡ラベル「4678」へ誘導されやすかったか？
'''
list_imbalanced_label = indices_imbalanced_label[0]
for i in list_imbalanced_label:
    print('index=%d, omomi=%0.6f' % (i, omomi_imbalanced_label[i]))

index=3, omomi=0.434943
index=4, omomi=0.423275
index=7, omomi=0.432055
index=8, omomi=0.410107
index=10, omomi=0.527233
index=13, omomi=0.446270
index=14, omomi=0.443815
index=16, omomi=0.421332
index=18, omomi=0.415889
index=22, omomi=0.423265
index=23, omomi=0.600548
index=24, omomi=0.662215
index=26, omomi=0.429349
index=28, omomi=0.469438
index=30, omomi=0.498986
index=33, omomi=0.411246
index=39, omomi=0.426341
index=42, omomi=0.488096
index=46, omomi=0.583862
index=49, omomi=0.434946
index=51, omomi=0.471685
index=55, omomi=0.459479
index=56, omomi=0.485934
index=57, omomi=0.538479
index=58, omomi=0.461232
index=59, omomi=0.442490
index=61, omomi=0.433435
index=65, omomi=0.455136
index=67, omomi=0.501318
index=69, omomi=0.423495
index=71, omomi=0.436998
index=73, omomi=0.519264
index=79, omomi=0.480239
index=83, omomi=0.474521
index=84, omomi=0.482857
index=87, omomi=0.555365
index=89, omomi=0.508266
index=91, omomi=0.400067
index=94, omomi=0.451061


### (3-3) featureから不均衡ラベルへの誘導ルートを確認

'ない' '出す' しかfeatureを持たない質問文でも、不均衡ラベル[4678]へ誘導されてしまう可能性があることの裏付けを取ります。

In [19]:
'''
    feature「ない」「出す」から、
    不均衡ラベル「4678」にいたるまで、
    どのユニットの経路を辿ったか？
    （そしてその重みは？）
'''
units = [u for u in list_imbalanced_label if u in list_nai]
for i in units:
    print('ない --(重み=%0.6f)--> Unit:%d --(重み=%0.6f)--> label:4678' % (omomi_nai[i], i, omomi_imbalanced_label[i]))

units = [u for u in list_imbalanced_label if u in list_dasu]
for i in units:
    print('出す --(重み=%0.6f)--> Unit:%d --(重み=%0.6f)--> label:4678' % (omomi_dasu[i], i, omomi_imbalanced_label[i]))

ない --(重み=0.783201)--> Unit:16 --(重み=0.421332)--> label:4678
出す --(重み=1.144951)--> Unit:83 --(重み=0.474521)--> label:4678
